In [1]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import torch.nn.functional as F
import tqdm

In [2]:
import argparse, warnings, sys
import torch

sys.path.append('/home/jh20/narin/dcase/DESED_task')
sys.path.append('/home/jh20/narin/dcase/DESED_task/recipes/dcase2022_task4_baseline')


parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--train_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/',     help='train data path')
parser.add_argument('--valid_data_folder',    default='/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/validation/',     help='validation data path')
parser.add_argument("--lr",default=1e-3, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=64, type=int)
parser.add_argument("--minibatchsize_valid", default=8, type=int)
parser.add_argument("--train_num_workers", default=6, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")
parser.add_argument("--seed", default=617, type=int)
parser.add_argument("--model_path", default='./hmd_CNN_outcome', type=str)
parser.add_argument("--logdir", default='./log_hmd_outcome/', type=str)
parser.add_argument("--model_name", default='hmd_CNN_outcome_model', type=str)
parser.add_argument("--start_iter", default=0, type=int)
parser.add_argument("--end_iter", default=70, type=int)
parser.add_argument("--gpu", default="0", type=str)

warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])

In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())

Available devices  1


In [4]:
from torch.utils.data import Dataset,DataLoader, RandomSampler
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
import librosa
import glob
import math






def read_recording(wav_fn, audio_length= 20, sr= 4000):

    recording, frequency = load_wav_file(wav_fn)


    length = audio_length*sr
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 

    return recording



class train_loader(Dataset):
    def __init__(self,data_folder, **kwargs):
        self.data_folder = data_folder
        
        self.wav_files = glob.glob(data_folder + '*.wav')
        self.num_wav_files = len(self.wav_files)
        
        self.murmur_classes = ['Present', 'Unknown', 'Absent']
        self.num_murmur_classes = len(self.murmur_classes)
        self.outcome_classes = ['Abnormal', 'Normal']
        self.num_outcome_classes = len(self.outcome_classes)
        
        self.age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
        self.recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

        
        
        examples = {}
        for filename in self.wav_files:
            current_patient_id = filename.split('/')[-1].split('_')[0]
            current_txt_file = self.data_folder + str(current_patient_id) + '.txt'
            with open(current_txt_file, 'r') as f:
                current_patient_data = f.read()
            current_loc = filename.split('/')[-1].split('.')[0].split('_')[1]
            if filename.split('/')[-1].split('.')[0]  not in examples.keys():
                current_wav_data = filename.split('/')[-1].split('.')[0]
                examples[current_wav_data] = {
                    "id" : current_patient_id,
                    "txt_fn" : current_txt_file,   #txt,
                    "wav_fn" : filename,  #wav
                    "num_location": get_num_locations(current_patient_data),
                    "frequency" : get_frequency(current_patient_data),
                    "location" : current_loc,
                    "age" : get_age(current_patient_data),
                    "sex" : get_sex(current_patient_data),
                    "height" : get_height(current_patient_data),
                    "weight" : get_weight(current_patient_data),
                    "pregnancy_status" : get_pregnancy_status(current_patient_data),
                    "murmur" : get_murmur(current_patient_data),
                    "outcome" : get_outcome(current_patient_data)

                }
        self.examples = examples
        self.examples_list = list(examples.keys())
        
        
        if len(self.examples_list) != self.num_wav_files:
            raise ValueError('length is different!')
    def __len__(self):
        return len(self.examples_list)
    
        
    def __getitem__(self, index):
        
        # patient info
        current_wav_dict = self.examples[self.examples_list[index]]
        current_patient_data = load_patient_data(current_wav_dict["txt_fn"])
        current_recording = read_recording(current_wav_dict["wav_fn"])
        
        #log mel feature
        log_mel_feature = librosa.power_to_db(librosa.feature.melspectrogram(y = current_recording.astype(np.float32),
                                                         sr= 4000,
                                                         n_mels=128,
                                                         n_fft=400, 
                                                         hop_length=128, 
                                                         win_length=400))
        
        # age
        current_patient_age = current_wav_dict["age"]
        current_age_group = np.zeros(6, dtype=np.float32)
        if current_patient_age in self.age_classes:
            j = self.age_classes.index(current_patient_age)
            current_age_group[j] = 1.0
        else :
            current_age_group[5] = 1.0
            
            
        # sex
        sex = current_wav_dict["sex"]
        sex_feature = np.zeros(2, dtype=np.float32)
        if compare_strings(sex, 'Female'):
            sex_feature[0] = 1.0
        elif compare_strings(sex, 'Male'):
            sex_feature[1] = 1.0
            
        # height and weight.
        height = current_wav_dict["height"]
        weight = current_wav_dict["weight"]
        
        ## simple impute
        if math.isnan(height) :
            height = 110.846  #mean
        if math.isnan(weight) :
            weight = 23.767   #mean
            
        height_weight = np.array([height, weight], dtype=np.float32)
        
        
        # Extract pregnancy
        preg_feature = np.zeros(2, dtype=np.float32)
        is_pregnant = current_wav_dict["pregnancy_status"]
        if is_pregnant == True:
            preg_feature[0] = 1.0
        elif is_pregnant == False:
            preg_feature[1] = 1.0

        # Extract location
        location = current_wav_dict["location"]
        num_recording_locations = len(self.recording_locations)
        loc_feature = np.zeros(num_recording_locations, dtype=np.float32)
        if location in self.recording_locations:
            j = self.recording_locations.index(location)
            loc_feature[j] = 1.0


        

        # label
        
        current_murmur = np.zeros(self.num_murmur_classes, dtype=np.float32)
        murmur = get_murmur(current_patient_data)
        if murmur in self.murmur_classes:
            j = self.murmur_classes.index(murmur)
            current_murmur[j] = 1


        current_outcome = np.zeros(self.num_outcome_classes, dtype=np.float32)
        outcome = get_outcome(current_patient_data)
        if outcome in self.outcome_classes:
            j = self.outcome_classes.index(outcome)
            current_outcome[j] = 1
            


        
        return log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome


In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

os.makedirs(args.model_path, exist_ok=True)
os.makedirs(args.logdir, exist_ok=True)

In [6]:
open(args.logdir + 'score_outcome.txt', "a+")
score_file = open(args.logdir + 'score_outcome.txt', "a+")

In [7]:
import logging
def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger

In [8]:
model_path = args.model_path
log_dir = args.logdir
logger = get_logger(log_dir + '/' + model_path)

In [9]:
import torch.nn as nn
import torch


class GLU(nn.Module):
    def __init__(self, input_num):
        super(GLU, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Linear(input_num, input_num)

    def forward(self, x):
        lin = self.linear(x.permute(0, 2, 3, 1))
        lin = lin.permute(0, 3, 1, 2)
        sig = self.sigmoid(x)
        res = lin * sig
        return res


class ContextGating(nn.Module):
    def __init__(self, input_num):
        super(ContextGating, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.linear = nn.Linear(input_num, input_num)

    def forward(self, x):
        lin = self.linear(x.permute(0, 2, 3, 1))
        lin = lin.permute(0, 3, 1, 2)
        sig = self.sigmoid(lin)
        res = x * sig
        return res
    
class Dense(nn.Module):
    
    def __init__(self,length):
        super(Dense, self).__init__()
        h1 = nn.Linear(length, 64)
        h2 = nn.Linear(64, 32)
        h3 = nn.Linear(32, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1,1)


class CNN(nn.Module):
    def __init__(
        self,
        n_in_channel=1,
        activation="Relu",
        conv_dropout=0,
        kernel_size=[3, 3, 3, 3, 3, 3],
        padding=[1, 1, 1, 1, 1, 1],
        stride=[1, 1, 1, 1, 1, 1],
        nb_filters=[16, 32, 64, 128, 256, 256],
        pooling=[(2, 2), (2, 2), (1, 2), (1, 2), (1, 2), (1, 2)],
        normalization="batch",
        **transformer_kwargs
    ):
        """
            Initialization of CNN network s
        
        Args:
            n_in_channel: int, number of input channel
            activation: str, activation function
            conv_dropout: float, dropout
            kernel_size: kernel size
            padding: padding
            stride: list, stride
            nb_filters: number of filters
            pooling: list of tuples, time and frequency pooling
            normalization: choose between "batch" for BatchNormalization and "layer" for LayerNormalization.
        """
        super(CNN, self).__init__()

        self.nb_filters = nb_filters
        cnn = nn.Sequential()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(264,132)
        self.fc2 = nn.Linear(132,2)
        self.fc_age = nn.Linear(6,3)
        self.fc_sex = nn.Linear(2,1)
        self.fc_hw = nn.Linear(2,1)
        self.fc_preg = nn.Linear(2,1)
        self.fc_loc = nn.Linear(5,2)
        

        def conv(i, normalization="batch", dropout=None, activ="relu"):
            nIn = n_in_channel if i == 0 else nb_filters[i - 1]
            nOut = nb_filters[i]
            cnn.add_module(
                "conv{0}".format(i),
                nn.Conv2d(nIn, nOut, kernel_size[i], stride[i], padding[i]),
            )
            if normalization == "batch":
                cnn.add_module(
                    "batchnorm{0}".format(i),
                    nn.BatchNorm2d(nOut, eps=0.001, momentum=0.99),
                )
            elif normalization == None:
                pass
                
            elif normalization == "layer":
                cnn.add_module("layernorm{0}".format(i), nn.GroupNorm(1, nOut))

            if activ.lower() == "leakyrelu":
                cnn.add_module("relu{0}".format(i), nn.LeakyReLU(0.2))
            elif activ.lower() == "relu":
                cnn.add_module("relu{0}".format(i), nn.ReLU())
            elif activ.lower() == "glu":
                cnn.add_module("glu{0}".format(i), GLU(nOut))
            elif activ.lower() == "cg":
                cnn.add_module("cg{0}".format(i), ContextGating(nOut))

            if dropout is not None:
                cnn.add_module("dropout{0}".format(i), nn.Dropout(dropout))
            elif dropout is None:
                pass

        # 128x862x64
        for i in range(len(nb_filters)):
            conv(i, normalization=normalization, dropout=conv_dropout, activ=activation)
            cnn.add_module(
                "pooling{0}".format(i), nn.MaxPool2d(pooling[i])
            )  # bs x tframe x mels

        self.cnn = cnn

    def forward(self, mel, age, sex, hw, preg, loc):
        """
        Forward step of the CNN module
        Args:
            x (Tensor): input batch of size (batch_size, n_channels, n_frames, n_freq)
        Returns:
            Tensor: batch embedded
        """
        # conv features
        mel = mel.unsqueeze(1)
        mel = self.cnn(mel)
        mel = self.avgpool(mel)
        mel = mel.view(mel.size(0), -1)
#         print(mel.shape)
        age = self.fc_age(age)
#         print(age.shape)
        sex = self.fc_sex(sex)
#         print(sex.shape)
        hw = self.fc_hw(hw)
#         print(hw.shape)
        preg = self.fc_preg(preg)
#         print(preg.shape)
        loc = self.fc_loc(loc)
#         print(loc.shape)
        x = torch.concat([mel,age,sex,hw,preg,loc],axis=1)
#         print(x.shape)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x =self.fc2(x)
        return x

In [10]:
cnn = CNN()
cnn = cnn.cuda(device)

In [11]:
from torchmetrics import F1
from torchmetrics import Accuracy

In [12]:
accuracy = Accuracy(num_classes=3, multiclass=True)
f1 = F1(num_classes=3,average='micro', multiclass=True)

In [13]:
opt = torch.optim.Adam(cnn.parameters(), lr=args.lr, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.91)
criterion = torch.nn.CrossEntropyLoss()

In [14]:
trainset = train_loader(args.train_data_folder)
validset = train_loader(args.valid_data_folder)
point_sampler = RandomSampler(trainset)

In [15]:
dataloader_train = DataLoader(dataset=trainset,
                        batch_size=args.minibatchsize_train,
                               sampler =point_sampler,
                               num_workers=args.train_num_workers)
dataloader_valid = DataLoader(dataset=validset,
                        batch_size=args.minibatchsize_valid,
                               num_workers=args.dev_num_workers)

In [16]:
all_file_train = len(trainset)
dataloader_train_len = len(dataloader_train)
output_preds_labels_train = torch.empty((all_file_train,))
output_preds_probs_train = torch.empty((all_file_train,2))
output_gt_labels_train = torch.empty((all_file_train,))
for iter_ in range(args.end_iter):
    start_time = time.time()
    running_loss = 0.0
    running_f1 = 0.0
    running_acc = 0.0
    idx = 0
    cnn.train()
    for log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome in tqdm.tqdm(dataloader_train):
        log_mel_feature = log_mel_feature.cuda()
        current_age_group = current_age_group.cuda()
        sex_feature = sex_feature.cuda()
        height_weight = height_weight.cuda()
        preg_feature = preg_feature.cuda()
        loc_feature = loc_feature.cuda()
        current_murmur = current_murmur.cuda()
        current_outcome = current_outcome.cuda()
        
        
        
        
        output_train = cnn(log_mel_feature,current_age_group,sex_feature,height_weight, preg_feature,loc_feature)
        loss = criterion(output_train, current_outcome)
        
        
        
        output_probs_train = F.log_softmax(output_train, dim=1)
        
        _, output_label_argmax = torch.max(output_probs_train, dim=1)
        
        
        output_label_argmax = output_label_argmax.data.detach().cpu()

        output_preds_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train]= output_label_argmax  #murmur 예측 레이블 (0,1,2)
        
        output_preds_probs_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train,:] = output_probs_train
        
        _, outcome_label_argmax = torch.max(current_outcome.data, dim=1)  #murmur groud truth (0,1,2)
        
        output_gt_labels_train[idx*args.minibatchsize_train:(idx+1)*args.minibatchsize_train] = outcome_label_argmax.data.detach().cpu()
        


        idx +=1              

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_loss += loss.item()

    train_f1 = f1(preds = torch.tensor(output_preds_labels_train, dtype=torch.int32), target = torch.tensor(output_gt_labels_train, dtype=torch.int32))
    train_acc = accuracy(preds = torch.tensor(output_preds_labels_train, dtype=torch.int32), target = torch.tensor(output_gt_labels_train, dtype=torch.int32))

        
    scheduler.step()
    print("lr: ", opt.param_groups[0]['lr'])
    logger.info("Iteration:{0}, train loss = {1:.6f} ,train F1 = {2:.6f} ,train ACC = {3:.6f} ".format(iter_, 
                 running_loss/dataloader_train_len,train_f1,train_acc))
        
    # eval
    all_file_valid = len(validset)
    dataloader_valid_len = len(dataloader_valid)
    cnn.eval()

    with torch.no_grad():
        running_loss_dev = 0.0
        running_f1_dev = 0.0
        running_acc_dev = 0.0
        index = 0
        output_preds_labels_dev = torch.empty((all_file_valid,))
        output_preds_probs_dev = torch.empty((all_file_valid,2))
        output_gt_labels_dev= torch.empty((all_file_valid,))
        for log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, current_murmur, current_outcome in tqdm.tqdm(dataloader_valid):
            log_mel_feature_dev = log_mel_feature.cuda()
            current_age_group_dev = current_age_group.cuda()
            sex_feature_dev = sex_feature.cuda()
            height_weight_dev = height_weight.cuda()
            preg_feature_dev = preg_feature.cuda()
            loc_feature_dev = loc_feature.cuda()
            current_murmur_dev = current_murmur.cuda()
            current_outcome_dev = current_outcome.cuda()


            output_dev = cnn(log_mel_feature_dev,current_age_group_dev,sex_feature_dev,height_weight_dev,preg_feature_dev,loc_feature_dev)
            loss_dev = criterion(output_dev, current_outcome_dev)                   
            running_loss_dev += loss_dev.item()
            
            
            
            
            output_probs_dev = F.log_softmax(output_dev, dim=1)
            
            _, output_label_argmax_dev = torch.max(output_probs_dev, dim=1)
            
            
            output_label_argmax_dev = output_label_argmax_dev.data.detach().cpu()

            output_preds_labels_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid]= output_label_argmax_dev  #outcome 예측 레이블 (0,1,2)
            
            output_preds_probs_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid,:] = output_probs_dev
            
            _, outcome_label_argmax_dev = torch.max(current_outcome_dev.data, dim=1)  #outcome groud truth (0,1,2)
            
            output_gt_labels_dev[index*args.minibatchsize_valid:(index+1)*args.minibatchsize_valid] = outcome_label_argmax_dev

            index = index+1           

        
        dev_f1 = f1(preds = torch.tensor(output_preds_labels_dev, dtype=torch.int32), target = torch.tensor(output_gt_labels_dev, dtype=torch.int32))
        dev_acc = accuracy(preds = torch.tensor(output_preds_labels_dev, dtype=torch.int32),target =  torch.tensor(output_gt_labels_dev, dtype=torch.int32))
            
            


            

    logger.info("valid Epoch:%d, Train loss=%.4f, Valid loss=%.4f,Valid F1=%.4f,Valid ACC=%.4f" %( iter_,
            running_loss/dataloader_train_len, running_loss_dev/dataloader_valid_len, dev_f1, dev_acc))
    score_file.write("%d epoch, VALID LOSS %f, VALID F1 %2.2f%% VALID ACC %2.2f%%\n"%(iter_, 
                            running_loss_dev/dataloader_valid_len, dev_f1, dev_acc))
    score_file.flush()
    
    torch.save(cnn.state_dict(), os.path.join(model_path, "{}_{}.model".format(args.model_name, iter_)))
    
    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))
    logger.info("*" * 50)

100%|███████████████████████████████████████████| 40/40 [00:15<00:00,  2.52it/s]

lr:  0.00091
07-18 14:00 [MainProcess, 24686] [INFO ]  Iteration:0, train loss = 0.710188 ,train F1 = 0.540284 ,train ACC = 0.540284 



100%|███████████████████████████████████████████| 79/79 [00:04<00:00, 17.18it/s]

07-18 14:00 [MainProcess, 24686] [INFO ]  valid Epoch:0, Train loss=0.7102, Valid loss=0.6366,Valid F1=0.6767,Valid ACC=0.6767
07-18 14:00 [MainProcess, 24686] [INFO ]  Time used for each epoch training: 20.468891859054565 seconds.
07-18 14:00 [MainProcess, 24686] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 40/40 [00:17<00:00,  2.33it/s]

...

07-18 14:25 [MainProcess, 24686] [INFO ]  valid Epoch:68, Train loss=0.4075, Valid loss=0.7482,Valid F1=0.6244,Valid ACC=0.6244
07-18 14:25 [MainProcess, 24686] [INFO ]  Time used for each epoch training: 21.99181842803955 seconds.
07-18 14:25 [MainProcess, 24686] [INFO ]  **************************************************



100%|███████████████████████████████████████████| 40/40 [00:17<00:00,  2.35it/s]

lr:  1.357992784773339e-06
07-18 14:26 [MainProcess, 24686] [INFO ]  Iteration:69, train loss = 0.405564 ,train F1 = 0.815166 ,train ACC = 0.815166 



100%|███████████████████████████████████████████| 79/79 [00:04<00:00, 16.20it/s]

07-18 14:26 [MainProcess, 24686] [INFO ]  valid Epoch:69, Train loss=0.4056, Valid loss=0.8049,Valid F1=0.6101,Valid ACC=0.6101
07-18 14:26 [MainProcess, 24686] [INFO ]  Time used for each epoch training: 21.90232753753662 seconds.
07-18 14:26 [MainProcess, 24686] [INFO ]  **************************************************


In [1]:
patient_data = load_patient_data(patient_files[1])
recordings = load_recordings(args.train_data_folder, patient_data)

NameError: name 'load_patient_data' is not defined